In [40]:
import os

def print_directory_contents(path, prefix='', ignore_items=None):
    if ignore_items is None:
        ignore_items = []

    # Mendapatkan daftar item dalam direktori
    items = os.listdir(path)
    # Mengurutkan item untuk konsistensi
    items.sort()
    
    for index, item in enumerate(items):
        # Menentukan jalur lengkap item
        item_path = os.path.join(path, item)
        # Menentukan apakah item adalah direktori
        is_last = index == len(items) - 1
        
        # Mengabaikan folder atau file yang ditentukan
        if item in ignore_items:
            continue
        
        # Mencetak nama item dengan format yang sesuai
        if os.path.isdir(item_path):
            print(f"{prefix}{'└── ' if is_last else '├── '}{item}/")
            # Memanggil fungsi secara rekursif untuk item dalam direktori
            print_directory_contents(item_path, prefix + ('    ' if is_last else '│   '), ignore_items)
        else:
            print(f"{prefix}{'└── ' if is_last else '├── '}{item}")

print('Struktur folder adalah sebagai berikut:')
print('\nimagetools')
ignore_list = ['.git', '.idea', 'uploaded', '.ipynb_checkpoints', 'scan.ipynb']
print_directory_contents('./', ignore_items=ignore_list)

def scan_files_recursive(directory, ignore_folders=None, ignore_files=None):
    if ignore_folders is None:
        ignore_folders = []
    if ignore_files is None:
        ignore_files = []

    found_files = []  # Daftar untuk menyimpan file yang ditemukan

    for root, dirs, files in os.walk(directory):
        # Menghapus folder yang ada dalam daftar ignore_folders dari dirs
        dirs[:] = [d for d in dirs if d not in ignore_folders]
        
        # Menghapus file yang ada dalam daftar ignore_files dari files
        files = [f for f in files if f not in ignore_files]
        
        for file in files:
            file_path = os.path.join(root, file)
            found_files.append(file_path)  # Menambahkan file ke daftar

    return found_files  # Mengembalikan daftar file yang ditemukan

# Daftar folder dan file yang ingin diabaikan
ignore_folders = ['.git', 'uploaded', '.idea', '.ipynb_checkpoints']
ignore_files = ['README.md', 'scan.ipynb', '.gitignore']  # Tambahkan file yang ingin diabaikan

# Memanggil fungsi dan menyimpan hasilnya
result = scan_files_recursive('.', ignore_folders, ignore_files)

# Menampilkan hasil dan isi file
for file in result:
    print(f'Isi dari file {file} adalah sebagai berikut:\n')
    try:
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
            print(content)  # Mencetak isi file
    except Exception as e:
        print(f'Tidak dapat membaca file {file}: {e}')
    print("\n" + "="*40 + "\n")  # Pemisah antar file

Struktur folder adalah sebagai berikut:

imagetools
├── .gitignore
├── README.md
├── admin/
│   ├── Dockerfile
│   ├── requirements.txt
│   └── setup_rabbitmq.py
├── docker-compose.yml
├── fastapi/
│   ├── Dockerfile
│   ├── main.py
│   └── requirements.txt
Isi dari file .\docker-compose.yml adalah sebagai berikut:

version: '3.8'

services:
  rabbitmq:
    image: rabbitmq:3.9.4-management
    container_name: imagetools_rabbitmq
    ports:
      - 5672:5672
      - 15672:15672
    environment:
      - RABBITMQ_DEFAULT_USER=guest
      - RABBITMQ_DEFAULT_PASS=guest
    networks:
      - rabbitmq_net

  admin:
    build:
      context: admin
    container_name: imagetools_admin
    depends_on:
      - rabbitmq
    networks:
      - rabbitmq_net

  fastapi:
    build:
      context: ./fastapi
    container_name: imagetools_fastapi
    ports:
      - 8000:8000
    volumes:
      - ./uploaded:/app/uploaded
    environment:
      - RABBITMQ_HOST=rabbitmq
      - RABBITMQ_USER=guest
      - R